# Pipeline E2E de Validación — Cleaning → Aggregation → Preprocessing (Grupo 04)

Este notebook valida la calidad de los datos procesados y prueba las funciones implementadas en:
- `src/preprocess/cleaning.py`
- `src/preprocess/aggregation.py`
- `src/preprocess/preprocessing.py`

Checklist de esta ejecución:
- [ ] Cargar raw y ejecutar todas las funciones de cleaning, guardando parquet/csv limpios
- [ ] Leer limpio y ejecutar aggregation (marca x día) + KPIs + calendario
- [ ] Probar utilidades de preprocessing: selección de serie, ARIMA/Prophet/ML/LSTM/SARIMAX
- [ ] Mostrar métricas de calidad y formas intermedias

Nota: este notebook asume que el archivo raw existe en `../data/raw/data_sample.parquet` (desde `notebooks/`).

In [15]:
# ===============
# SETUP & IMPORTS
# ===============
import sys
from pathlib import Path
import pandas as pd
import numpy as np

# Asegurar que src/ esté en el path
ROOT = Path('..').resolve()
SRC = ROOT / 'src'
if str(SRC) not in sys.path:
    sys.path.append(str(SRC))

# Forzar recarga de módulos para reflejar cambios recientes en src/
import importlib
import preprocess.cleaning as _cleaning_mod
import preprocess.aggregation as _aggregation_mod
import preprocess.preprocessing as _preprocessing_mod
importlib.reload(_cleaning_mod)
importlib.reload(_aggregation_mod)
importlib.reload(_preprocessing_mod)

from preprocess.cleaning import (
    CleaningConfig,
    run_cleaning,
    load_raw,
    normalize_strings_and_placeholders,
    parse_dates,
    normalize_brand,
    drop_high_missing_columns,
    drop_all_missing_rows,
    coerce_types,
    drop_duplicates,
    negatives_to_nan,
    impute_missing,
    handle_outliers,
    drop_constant_cols,
    reduce_categoricals_cardinality,
    DataQualityReport,
)

from preprocess.aggregation import (
    calculate_day_brand_aggregation,
    generate_day_brand_grouped_data_files,   
)
from preprocess.preprocessing import (
    TSConfig,
    TimeSeriesPreprocessor,
)

RAW = ROOT / 'data' / 'raw' / 'data_sample.parquet'
CLEAN_PARQUET = ROOT / 'data' / 'raw' / 'data_sample_cleaned_group04.parquet'
BRAND_DAILY_PARQUET = ROOT / 'data' / 'raw' / 'brand_daily_group04.parquet'

print('ROOT:', ROOT)
print('RAW exists:', RAW.exists())

ROOT: C:\ProyectoParcialGrupo4\pc1_20252_metodologia_data_science
RAW exists: True


In [16]:
# ============================
# 1) CLEANING — Paso a paso (proteger parsed_date y product_brand)
# ============================

cfg_clean = CleaningConfig(
    raw_path=str(RAW),
    out_parquet=str(CLEAN_PARQUET),
    out_csv=str(CLEAN_PARQUET).replace('.parquet', '.csv'),
    persist=True,
    # Para columnas con alto % de nulos: imputar (num->media, cat->moda)
    drop_columns_missing_action='impute',
)

# Cargar raw
raw_df = load_raw(cfg_clean)
print('Raw shape:', raw_df.shape)

# Reporte de calidad acumulado
report = DataQualityReport()

# 1) Normalizar strings/placeholders
step_df = normalize_strings_and_placeholders(raw_df, cfg_clean)
print('After normalize_strings:', step_df.shape)

# 2) Parseo de fechas -> crea 'parsed_date' si hay candidata
step_df = parse_dates(step_df, cfg_clean)
print("Has 'parsed_date'?:", 'parsed_date' in step_df.columns)

# 3) Normalizar marca (si falta -> Unknown) y crear columna canónica 'product_brand'
step_df = normalize_brand(step_df, cfg_clean, report)
# Fallback robusto por si el kernel tiene una versión antigua del módulo
if 'product_brand' not in step_df.columns:
    for cand in ['product_brand','productBrand','brand','Brand','product_brand_name']:
        if cand in step_df.columns:
            step_df['product_brand'] = step_df[cand].astype('object')
            break
    else:
        step_df['product_brand'] = 'Unknown'
print("Has 'product_brand'?:", 'product_brand' in step_df.columns)

# 4) Gestionar columnas con muchos nulos (imputación configurada) — respeta columnas protegidas
step_df = drop_high_missing_columns(step_df, cfg_clean, report)
print('High-missing handled. Dropped:', report.dropped_columns_missing, '| Imputed:', getattr(report, 'imputed_columns_high_missing', []))

# 5) Drop filas completamente nulas
step_df = drop_all_missing_rows(step_df, cfg_clean, report)
print('Dropped all-missing rows:', report.dropped_rows_all_missing)

# 6) Coerción de tipos
step_df = coerce_types(step_df, cfg_clean)

# 7) Duplicados
step_df = drop_duplicates(step_df, cfg_clean, report)
print('Duplicates dropped:', report.duplicate_rows_dropped)

# 8) Negativos -> NaN
step_df = negatives_to_nan(step_df, cfg_clean, report)
print('Negatives→NaN cols:', report.negatives_to_nan_cols)

# 9) Imputación
step_df = impute_missing(step_df, cfg_clean)

# 10) Winsorize (outliers)
step_df = handle_outliers(step_df, cfg_clean, report)
print('Winsorized cols:', report.winsorized_cols[:8], '...')

# 11) Columnas constantes (respeta columnas protegidas)
step_df = drop_constant_cols(step_df, cfg_clean, report)
print('Constant cols dropped:', report.constant_cols_dropped)

# 12) Reducir cardinalidad (categorías raras -> 'Otros') — no afecta columnas protegidas
step_df = reduce_categoricals_cardinality(step_df, cfg_clean)

# Sanity check: la marca y fecha deben existir para el pipeline Día/Marca
assert 'product_brand' in step_df.columns, "ERROR: product_brand fue eliminada"
assert 'parsed_date' in step_df.columns, "ERROR: parsed_date no existe; verifica columnas de fecha en raw o CleaningConfig.date_candidates"

# Persistir y cerrar reporte
report.n_rows_before, report.n_cols_before = raw_df.shape
report.n_rows_after, report.n_cols_after = step_df.shape
print('Cleaned shape:', step_df.shape)

# Guardar
step_df.to_parquet(cfg_clean.out_parquet, index=False)
print('Saved parquet:', CLEAN_PARQUET.exists())

# Vista rápida
display(step_df.head())
display(report.to_frame())

# Ver todas las columnas restantes
import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 220)
print(f"✅ Columnas finales ({step_df.shape[1]}):")
print(list(step_df.columns))

Raw shape: (74457, 77)
After normalize_strings: (74457, 77)
Has 'parsed_date'?: True
After normalize_strings: (74457, 77)
Has 'parsed_date'?: True
Has 'product_brand'?: True
Has 'product_brand'?: True


C:\ProyectoParcialGrupo4\pc1_20252_metodologia_data_science\src\preprocess\cleaning.py:285: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  out[c] = ser.fillna(fillv)


High-missing handled. Dropped: [] | Imputed: ['is_impression', 'is_click', 'promo_id', 'promo_name', 'promo_creative', 'promo_position', 'bounces', 'campaign', 'keyword', 'ad_content', 'adwords_campaign_id', 'adwords_adgroup_id', 'adwords_creative_id', 'adwords_criteria_id', 'gclid', 'ad_network_type', 'is_entrance', 'referer', 'entrance_page_path']
Dropped all-missing rows: 0
Dropped all-missing rows: 0
Duplicates dropped: 240
Duplicates dropped: 240
Negatives→NaN cols: []
Negatives→NaN cols: []


C:\ProyectoParcialGrupo4\pc1_20252_metodologia_data_science\src\preprocess\cleaning.py:355: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  out[c] = out[c].fillna(fillv)


Winsorized cols: ['transaction_revenue_usd', 'transaction_tax_usd', 'transaction_shipping_usd', 'product_quantity', 'product_price_usd', 'product_revenue_usd', 'visitor_id', 'session_id'] ...
Constant cols dropped: ['transaction_affiliation', 'currency_code', 'is_impression', 'is_click', 'promo_id', 'promo_name', 'promo_creative', 'promo_position', 'total_visits', 'bounces', 'new_visits', 'is_true_direct', 'browser_version', 'os_version', 'mobile_device_brand', 'mobile_device_model', 'device_language', 'screen_resolution', 'adwords_campaign_id', 'adwords_adgroup_id', 'adwords_creative_id', 'adwords_criteria_id', 'ad_network_type', 'is_interaction', 'is_entrance', 'is_exit', 'referer', 'page_path', 'hostname', 'entrance_page_path', 'exit_page_path', 'social_engagement_type']
Constant cols dropped: ['transaction_affiliation', 'currency_code', 'is_impression', 'is_click', 'promo_id', 'promo_name', 'promo_creative', 'promo_position', 'total_visits', 'bounces', 'new_visits', 'is_true_direct

,transaction_date,parsed_date,transaction_id,transaction_revenue_usd,transaction_tax_usd,transaction_shipping_usd,product_sku,product_name,product_category,product_brand,product_variant,product_quantity,product_price_usd,product_revenue_usd,visitor_id,session_id,session_number,session_start_time,total_hits,total_pageviews,time_on_site_seconds,traffic_source,traffic_medium,campaign,keyword,ad_content,referral_path,channel_grouping,browser,operating_system,is_mobile,device_category,continent,sub_continent,country,region,metro,city,network_domain,gclid,hit_number,hit_time_ms,hit_hour,hit_minute,page_title
0,20170801,2017-08-01 00:00:00+00:00,ORD201708011814,96.32,13.11,13.0,GGOEGAEJ028013,Google Women's Short Sleeve Hero Tee Grey,Apparel,(not set),SM,1.0,5.10,19.49,509972280802528263,1501522116,4,1501522116,9,9,165.0,(direct),(none),AW - Dynamic Search Ads Whole Site,6qEhsCssdK0z36ri,Google Merchandise Collection,/,Display,Chrome,Macintosh,False,desktop,Americas,Northern America,United States,California,San Francisco-Oakland-San Jose CA,Mountain View,comcast.net,CJ3Ls5Diqs4CFQ6oaQodXegHOA,8,155519,14,0,Checkout Confirmation
1,20170801,2017-08-01 00:00:00+00:00,ORD201708011814,40.29,8.82,11.0,GGOEGAEJ028013,Google Women's Short Sleeve Hero Tee Grey,Apparel,(not set),SM,2.0,5.10,11.45,509972280802528263,1501522116,4,1501522116,9,9,165.0,(direct),(none),AW - Dynamic Search Ads Whole Site,6qEhsCssdK0z36ri,Google Merchandise Collection,/,Display,Chrome,Macintosh,False,desktop,Americas,Northern America,United States,California,San Francisco-Oakland-San Jose CA,Mountain View,comcast.net,CJ3Ls5Diqs4CFQ6oaQodXegHOA,8,155519,14,0,Checkout Confirmation
2,20170801,2017-08-01 00:00:00+00:00,ORD201708011814,96.32,13.11,13.0,GGOEGALB034113,Google Women's Vintage Hero Tee Black,Apparel,(not set),SM,1.0,5.70,19.49,509972280802528263,1501522116,4,1501522116,9,9,165.0,(direct),(none),AW - Dynamic Search Ads Whole Site,6qEhsCssdK0z36ri,Google Merchandise Collection,/,Display,Chrome,Macintosh,False,desktop,Americas,Northern America,United States,California,San Francisco-Oakland-San Jose CA,Mountain View,comcast.net,CJ3Ls5Diqs4CFQ6oaQodXegHOA,8,155519,14,0,Checkout Confirmation
3,20170801,2017-08-01 00:00:00+00:00,ORD201708011814,40.29,8.82,11.0,GGOEGALB034113,Google Women's Vintage Hero Tee Black,Apparel,(not set),SM,2.0,5.70,12.65,509972280802528263,1501522116,4,1501522116,9,9,165.0,(direct),(none),AW - Dynamic Search Ads Whole Site,6qEhsCssdK0z36ri,Google Merchandise Collection,/,Display,Chrome,Macintosh,False,desktop,Americas,Northern America,United States,California,San Francisco-Oakland-San Jose CA,Mountain View,comcast.net,CJ3Ls5Diqs4CFQ6oaQodXegHOA,8,155519,14,0,Checkout Confirmation
4,20170801,2017-08-01 00:00:00+00:00,ORD201708011814,96.32,13.11,13.0,GGOEGOCB017499,Leatherette Journal,Office,(not set),Single Option Only,1.0,7.69,19.49,509972280802528263,1501522116,4,1501522116,9,9,165.0,(direct),(none),AW - Dynamic Search Ads Whole Site,6qEhsCssdK0z36ri,Google Merchandise Collection,/,Display,Chrome,Macintosh,False,desktop,Americas,Northern America,United States,California,San Francisco-Oakland-San Jose CA,Mountain View,comcast.net,CJ3Ls5Diqs4CFQ6oaQodXegHOA,8,155519,14,0,Checkout Confirmation


,n_rows_before,n_cols_before,n_rows_after,n_cols_after,dropped_columns_missing,dropped_rows_all_missing,duplicate_rows_dropped,negatives_to_nan_cols,winsorized_cols,constant_cols_dropped,brand_unknown_rows
0,74457,77,74217,45,[],0,240,[],"[transaction_revenue_usd, transaction_tax_usd,...","[transaction_affiliation, currency_code, is_im...",0


✅ Columnas finales (45):
['transaction_date', 'parsed_date', 'transaction_id', 'transaction_revenue_usd', 'transaction_tax_usd', 'transaction_shipping_usd', 'product_sku', 'product_name', 'product_category', 'product_brand', 'product_variant', 'product_quantity', 'product_price_usd', 'product_revenue_usd', 'visitor_id', 'session_id', 'session_number', 'session_start_time', 'total_hits', 'total_pageviews', 'time_on_site_seconds', 'traffic_source', 'traffic_medium', 'campaign', 'keyword', 'ad_content', 'referral_path', 'channel_grouping', 'browser', 'operating_system', 'is_mobile', 'device_category', 'continent', 'sub_continent', 'country', 'region', 'metro', 'city', 'network_domain', 'gclid', 'hit_number', 'hit_time_ms', 'hit_hour', 'hit_minute', 'page_title']


In [17]:
# ============================
# 2) AGGREGATION — Marca x Día
# ============================

import importlib
import pandas as pd
from pathlib import Path

# Importar SIEMPRE el módulo actual (recarga para evitar versiones viejas en memoria)
import preprocess.aggregation as aggmod
aggmod = importlib.reload(aggmod)

# Rutas (asumimos que ya tienes estas variables definidas en tu notebook)
IN_PARQUET  = str(CLEAN_PARQUET)        # parquet limpio que viene de cleaning
OUT_PARQUET = str(BRAND_DAILY_PARQUET)  # parquet agregado día/marca de salida

print("Using cleaned input:", IN_PARQUET)
print("Saving to          :", OUT_PARQUET)

# -------------------------------------------------------------
# 1) Probar calculate_day_brand_aggregation directamente (en memoria)
# -------------------------------------------------------------
# Cargar el parquet limpio
df_in = pd.read_parquet(IN_PARQUET)

# Asegurar que exista 'transaction_date' (si sólo tienes 'parsed_date', lo copiamos)
if "transaction_date" not in df_in.columns and "parsed_date" in df_in.columns:
    df_in = df_in.copy()
    df_in["transaction_date"] = df_in["parsed_date"]

# Asegurar que exista 'product_brand'
if "product_brand" not in df_in.columns:
    # Trata de adivinar una columna de marca si el cleaning dejó otro nombre
    candidates = [c for c in df_in.columns if "brand" in c.lower()]
    if candidates:
        df_in = df_in.rename(columns={candidates[0]: "product_brand"})
    else:
        raise ValueError("No se encontró columna 'product_brand' ni ninguna similar en el parquet de entrada.")

# (Opcional) Si quieres contar eventos: metric_col=None
# (Si quieres sumar una métrica, pon el nombre en metric_col, p.ej. 'product_quantity')
metric_col = None  # <-- cambia a 'product_quantity' o similar si quieres suma de métrica

agg_mem = aggmod.calculate_day_brand_aggregation(df_in, metric_col=metric_col)
print("Aggregated (in-memory) shape:", agg_mem.shape)
display(agg_mem.head())

# -------------------------------------------------------------
# 2) Probar generate_day_brand_grouped_data_files (lee y escribe parquet)
# -------------------------------------------------------------
# Usamos la función del módulo que lee desde IN_PARQUET y guarda en OUT_PARQUET
result = aggmod.generate_day_brand_grouped_data_files(
    in_path=IN_PARQUET,
    out_path=OUT_PARQUET,
    metric_col=metric_col
)
print("Writer result:", result, "| Exists:", Path(OUT_PARQUET).exists())

# Cargar lo que se guardó y comparar rápidamente con el cálculo en memoria
agg_disk = pd.read_parquet(OUT_PARQUET)
print("Aggregated (disk)  shape:", agg_disk.shape)

# Orden y columnas iguales para comparar
_cols = sorted(list(set(agg_mem.columns) | set(agg_disk.columns)))
cmp_ok = agg_mem.sort_values(_cols).reset_index(drop=True).equals(
    agg_disk.sort_values(_cols).reset_index(drop=True)
)
print("In-memory == On-disk ?", cmp_ok)

# -------------------------------------------------------------
# 3) Serie de una sola acción (filtrar la marca '(not set)' y dejar serie diaria)
# -------------------------------------------------------------
ONE_BRAND = "(not set)"  # tu única marca para el caso de “serie de una sola acción”
#events = número de registros (interacciones/ventas/sesiones, etc.) en un día para una marca.
#Para serie de una sola acción (una sola marca), usar la columna events agregada por 
#día te da una serie temporal univariada diaria: justo lo que necesitas para 
# “Predicción de performance diaria por marca”.
df_single = agg_disk[agg_disk["product_brand"] == ONE_BRAND].copy()

# Determinar el nombre de la columna de valor (events o sum_<metric>)
value_col = next((c for c in df_single.columns if c.startswith("sum_")), None)
if value_col is None:
    value_col = "events"  # cuando metric_col=None

# Asegurar datetime y set index a fecha
df_single["transaction_date"] = pd.to_datetime(df_single["transaction_date"], utc=True).dt.normalize()
ts = df_single.set_index("transaction_date")[value_col].sort_index()

print(f"\nSingle-action brand: {ONE_BRAND}")
print("Time series length:", len(ts), "| from", ts.index.min(), "to", ts.index.max())
display(ts.head(10))


Using cleaned input: C:\ProyectoParcialGrupo4\pc1_20252_metodologia_data_science\data\raw\data_sample_cleaned_group04.parquet
Saving to          : C:\ProyectoParcialGrupo4\pc1_20252_metodologia_data_science\data\raw\brand_daily_group04.parquet
Aggregated (in-memory) shape: (365, 3)
Aggregated (in-memory) shape: (365, 3)


,transaction_date,product_brand,events
0,2016-08-01 00:00:00+00:00,(not set),208
1,2016-08-02 00:00:00+00:00,(not set),124
2,2016-08-04 00:00:00+00:00,(not set),78
3,2016-08-05 00:00:00+00:00,(not set),376
4,2016-08-06 00:00:00+00:00,(not set),148


Writer result: success | Exists: True
Aggregated (disk)  shape: (365, 3)
In-memory == On-disk ? True

Single-action brand: (not set)
Time series length: 365 | from 2016-08-01 00:00:00+00:00 to 2017-08-01 00:00:00+00:00


transaction_date
2016-08-01 00:00:00+00:00    208
2016-08-02 00:00:00+00:00    124
2016-08-04 00:00:00+00:00     78
2016-08-05 00:00:00+00:00    376
2016-08-06 00:00:00+00:00    148
2016-08-07 00:00:00+00:00    122
2016-08-08 00:00:00+00:00    256
2016-08-09 00:00:00+00:00    300
2016-08-10 00:00:00+00:00    314
2016-08-11 00:00:00+00:00    426
Name: events, dtype: int64

In [21]:
# ============================
# 3) PREPROCESSING — Prueba con brand_daily_group04.parquet
# ============================

from pathlib import Path
import importlib
import pandas as pd

# Recarga por si editaste archivos
import preprocess.preprocessing as _preprocessing_mod
_ = importlib.reload(_preprocessing_mod)
from preprocess.preprocessing import TSConfig, TimeSeriesPreprocessor

# Ruta al parquet agregado
DATA_FOR_MODELS = str(Path('..') / 'data' / 'raw' / 'brand_daily_group04.parquet')

# 1) CONFIG: usa exactamente los nombres del parquet agregado
cfg = TSConfig(
    date_col='transaction_date',   # <- columna de fecha en el parquet agregado
    brand_col='product_brand',     # <- columna de marca
    target_metric='events',        # <- métrica objetivo (conteo diario)
    freq='D',
    fill_missing='zero'
)

prep = TimeSeriesPreprocessor(cfg)

# 2) Cargar datos (debe quedar indexado por fecha)
df_models = prep.load_data(DATA_FOR_MODELS)
print('Model DF shape:', df_models.shape)
print('Index dtype:', type(df_models.index))
print('Columns:', list(df_models.columns))

# 3) Elegir una marca (si sólo tienes "(not set)", será esa)
brand = "(not set)"
if 'product_brand' in df_models.columns:
    # si quieres forzar la única disponible:
    brand = df_models['product_brand'].mode(dropna=True).iloc[0]
print('Brand seleccionada:', brand)

# 4) Construir la serie diaria univariada (y = events)
series = prep.select_brand_series(df_models, brand=brand, metric=cfg.target_metric)
print('Series shape:', series.shape, '| rango:', series.index.min(), '→', series.index.max())
display(series.head(10))

# ============================
# 5) Pruebas de preparación por modelo
# ============================

# ARIMA / SARIMA (si no estacionaria, devolverá la diferenciada)
arima_series = prep.prepare_arima_data(series)
print('ARIMA series len:', len(arima_series))

# Prophet
prophet_df, exog_cols = prep.prepare_prophet_data(series)
print('Prophet df:', prophet_df.shape, '| exog:', exog_cols)
display(prophet_df.head())

# ML (features de lags/rolling, ejemplo de fechas arbitrarias ajusta si tu rango es distinto)
X_tr, y_tr, X_te, y_te = prep.prepare_ml_data(series,
                                              train_start=str(series.index.min().date()),
                                              train_end=str((series.index.min()+pd.Timedelta(days=250)).date()),
                                              test_start=str((series.index.min()+pd.Timedelta(days=251)).date()))
print('ML shapes:', X_tr.shape, y_tr.shape, X_te.shape, y_te.shape)

# LSTM (usa las mismas features que ML; secuencias)
feats = prep.create_features(series).dropna()
Xtr, Xte, ytr, yte = prep.prepare_lstm_data(feats, sequence_length=30)
print('LSTM shapes:', Xtr.shape, Xte.shape, ytr.shape, yte.shape)

# SARIMAX exógenas (ejemplo sin eventos externos)
exog = prep.create_sarimax_exogenous(series, events_periods=[])
print('Exogenous shape:', exog.shape)


Model DF shape: (365, 2)
Index dtype: <class 'pandas.core.indexes.datetimes.DatetimeIndex'>
Columns: ['product_brand', 'events']
Brand seleccionada: (not set)
Series shape: (366,) | rango: 2016-08-01 00:00:00+00:00 → 2017-08-01 00:00:00+00:00


Date
2016-08-01 00:00:00+00:00    208.0
2016-08-02 00:00:00+00:00    124.0
2016-08-03 00:00:00+00:00      0.0
2016-08-04 00:00:00+00:00     78.0
2016-08-05 00:00:00+00:00    376.0
2016-08-06 00:00:00+00:00    148.0
2016-08-07 00:00:00+00:00    122.0
2016-08-08 00:00:00+00:00    256.0
2016-08-09 00:00:00+00:00    300.0
2016-08-10 00:00:00+00:00    314.0
Freq: D, Name: events, dtype: float64

ARIMA series len: 366
Prophet df: (366, 2) | exog: []


,ds,y
0,2016-08-01,208.0
1,2016-08-02,124.0
2,2016-08-03,0.0
3,2016-08-04,78.0
4,2016-08-05,376.0


ML shapes: (151, 27) (151,) (115, 27) (115,)
LSTM shapes: (188, 30, 28) (48, 30, 28) (188,) (48,)
Exogenous shape: (366, 0)
